Utilizing engine functions to create a full experiment pipeline for reference on a dummy dataset.

In [1]:
# Import modules required for a train.py file
import os
import torch

import utils.engine as engine
import utils.data as data_setup

from torchvision import transforms

%load_ext autoreload
%autoreload 2

In [2]:
# Model Arch file, an example model is below
import torch
from torch import nn 

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=hidden_units, 
                    kernel_size=3, 
                    stride=1, 
                    padding=0),  
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units, 
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
      )
      self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from? 
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
      )

  def forward(self, x: torch.Tensor):
      x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x
      # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion


In [4]:
# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Create the writeer
writer = engine.create_writer(
    experiment_name="dummy_data",
    model_name="tinyvgg_model",
    extra="5_epochs"
)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device,
             writer=writer)

# Save the model with help from utils.py
engine.save_model(model=model,
                 target_dir="models",
                 model_name="exp_tinyvgg_model.pth")

[INFO] Created SummaryWriter, saving to: runs/2025-08-12/dummy_data/tinyvgg_model/5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Training Epoch 0:   0%|          | 0/8 [00:00<?, ?it/s]

Testing Epoch 0:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0967 | train_acc: 0.3867 | test_loss: 1.0990 | test_acc: 0.2604 | train_epoch_time: 1.0620 | test_epoch_time: 0.7693


...

Testing Epoch 3:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 4 | train_loss: 1.0871 | train_acc: 0.4648 | test_loss: 1.0790 | test_acc: 0.3816 | train_epoch_time: 0.9224 | test_epoch_time: 0.7303


Training Epoch 4:   0%|          | 0/8 [00:00<?, ?it/s]

Testing Epoch 4:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 5 | train_loss: 1.0877 | train_acc: 0.3359 | test_loss: 1.0270 | test_acc: 0.5521 | train_epoch_time: 0.9091 | test_epoch_time: 0.7571
[INFO] Saving model to: models/exp_tinyvgg_model.pth
